# Sentiment Analysis of Italian Restaurant Reviews

The project aims to analyze customer sentiment towards Italian restaurants in France and Italy. The analysis is based on textual reviews collected from two different data sources: OpenTable and Yelp.

By employing sentiment analysis, we aim to uncover trends and compare how Italian cuisine and the general quality of the related businesses are perceived across the two regions.

## Yelp Fusion API

In [1]:
import requests

# Set up API credentials
API_KEY = "3lX3EqE4bLsHCwaN8tyZ3kNNg_tykrIiw8cgEDcbNOeGYo9m22YYW5as-1dPp-f0Gy_X8_12CDEiqVgbM0SdKgKE2x94_w4-_PLu8Kfufdj-kBvbYWCGNmUUjyZHZ3Yx"
API_URL = "https://api.yelp.com/v3/businesses/search"

# Define the search parameters
headers = {"Authorization": f"Bearer {API_KEY}"}
params = {
    "term": "",
    "location": "Italy",
    "categories": "restaurants",
    "limit": 10,  # Number of results to fetch
}

# Make the request
response = requests.get(API_URL, headers=headers, params=params)

# Parse the response
if response.status_code == 200:
    data = response.json()
    for business in data.get("businesses", []):
        print(f"Name: {business['name']}")
        print(f"Rating: {business['rating']}")
        print(f"Address: {', '.join(business['location']['display_address'])}")
        print(f"Phone: {business.get('phone', 'N/A')}")
        print("-" * 40)
else:
    print(f"Error: {response.status_code} - {response.text}")


Name: 44 Osteria Quarantaquattro
Rating: 4.5
Address: Via Aureliana 42, 00187 Rome, Italy
Phone: +390642013318
----------------------------------------
Name: Life
Rating: 4.3
Address: Via della Vite 28, 00187 Rome, Italy
Phone: +390669380948
----------------------------------------
Name: Ai Tre Scalini
Rating: 4.4
Address: Via Panisperna 251, 00184 Rome, Italy
Phone: +390648907495
----------------------------------------
Name: Roscioli
Rating: 4.3
Address: Via dei Giubbonari 21, 00186 Rome, Italy
Phone: +39066875287
----------------------------------------
Name: Cantina e Cucina
Rating: 4.5
Address: Via del Governo Vecchio 87, 00186 Rome, Italy
Phone: +39066892574
----------------------------------------
Name: Il Chianti
Rating: 4.3
Address: Via del Lavatore 81, 00187 Rome, Italy
Phone: +39066787550
----------------------------------------
Name: Hostaria La Botticella
Rating: 4.5
Address: Vicolo del Leopardo 39A, 00153 Rome, Italy
Phone: +39065814738
-----------------------------------

## OpenTable

In [2]:
%pip install selenium --quiet

Note: you may need to restart the kernel to use updated packages.


In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def scroll_down_page(driver, speed=8):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

def scrape_opentable_restaurants(max_restaurants=10):
    
    # queries OpenTable restaurants in 'Paris' under 'Italian' cousine category, ordered by rating
    url = "https://www.opentable.com/s?term=paris&cuisineIds%5B%5D=48e9d049-40cf-4cb9-98d9-8c47d0d58986&sortBy=rating"
    
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    scroll_down_page(driver)
    time.sleep(3)

    restaurants = []

    try:
        # Extract review elements
        restaurant_elements = driver.find_elements(By.CLASS_NAME, 'qCITanV81-Y-')

        for restaurant_element in restaurant_elements:
            try:
                restaurant_name = restaurant_element.text # name
                restaurant_link = restaurant_element.get_attribute('href') # link
                restaurant_link = restaurant_link[:-122] # removed parameters
                restaurants.append({'restaurant_name': restaurant_name, 'restaurant_link': restaurant_link})

            except Exception as e:
                print(f"Error extracting restaurants: {e}")
                continue

    except Exception as e:
        print(f"Error during scraping: {e}")
    finally:
        driver.quit()

    return restaurants

def scrape_opentable_reviews(url, max_reviews=10):
    """
    Scrapes reviews from an OpenTable restaurant page.

    Args:
    - url (str): The URL of the restaurant's OpenTable page.
    - max_reviews (int): The maximum number of reviews to scrape.

    Returns:
    - list of dict: A list of reviews with the review text and date.
    """
    # Initialize WebDriver
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)

    reviews = []

    try:
        while len(reviews) < max_reviews:
            # Extract review elements
            review_elements = driver.find_elements(By.CLASS_NAME, 'afkKaa-4T28-')

            for review_element in review_elements:
                try:
                    review_text = review_element.find_element(By.CLASS_NAME, '_6rFG6U7PA6M-').text
                    review_date = review_element.find_element(By.CLASS_NAME, 'iLkEeQbexGs-').text
                    reviews.append({'review_text': review_text, 'review_date': review_date})

                    # Stop if we reach the maximum reviews
                    if len(reviews) >= max_reviews:
                        break
                except Exception as e:
                    print(f"Error extracting review: {e}")
                    continue

            # Attempt to click "Load More Reviews" if it exists
            try: # TODO: fix
                load_more_button = driver.find_element(By.CLASS_NAME, 'TkpxbcBbu80-')
                load_more_button.click()
                time.sleep(3)  # Wait for reviews to load
            except:
                break

    except Exception as e:
        print(f"Error during scraping: {e}")
    finally:
        driver.quit()

    return reviews

In [30]:
restaurants = scrape_opentable_restaurants()

print("restaurants count: " + str(len(restaurants)))

for restaurant in restaurants:
    print(f"Restaurant: {restaurant['restaurant_name']}")
    reviews = scrape_opentable_reviews(restaurant['restaurant_link'], max_reviews=15)
    
    print("count: " + str(len(reviews)))

    #for idx, review in enumerate(reviews, 1):
    #  print(f"Review {idx}:")
    #  print(f"Date: {review['review_date']}")
    #  print(f"Text: {review['review_text']}\n")

restaurants count: 30
Restaurant: Truffes Folies Paris 7
count: 10
Restaurant: Epoca Restaurant
count: 4
Restaurant: Penati al Baretto
NEW
count: 1
Restaurant: TCHELLO
count: 3
Restaurant: Restaurant Zo
count: 4
Restaurant: SEB Cafe
count: 3
Restaurant: Bistro L'Olivier
count: 10
Restaurant: Senza nome
count: 4
Restaurant: PEPEGUSTO
count: 2
Restaurant: Mamamia
count: 10
Restaurant: SOGNO PARIS
count: 10
Restaurant: Valentina
count: 2
Restaurant: Liberta
count: 0
Restaurant: Restaurant Conti
count: 0
Restaurant: Emporio Armani Caffè
count: 0
Restaurant: Maddy
count: 4
Restaurant: Baretto
count: 10
Restaurant: da Alfredo
count: 10
Restaurant: Giulia
count: 0
Restaurant: Bistrot Kani
count: 10
Restaurant: L’Attilio Paris
count: 2
Restaurant: Limone
count: 0
Restaurant: Lombardi
count: 4
Restaurant: Liberto
count: 2
Restaurant: Pepita Paris
NEW
count: 0
Restaurant: Mori Venice Bar
NEW
count: 0
Restaurant: Giusé Trattoria
count: 7
Restaurant: Armani Ristorante
count: 0
Restaurant: Laïa
cou